# Wikipedia Notable Life Expectancies

# [Notebook 4 of:  Data Cleaning Part 3](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean3_thanak_2022_06_23.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean2.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean2", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 132652 rows and 21 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,British dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,Irish economist,writer,and academic,None,None,None,None,None,None,None,68.0,None


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death
132650,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2,2022,June,(1980),None,Russian volleyball player,Olympic champion and coach,None,None,None,None,None,None,None,None,69.0,None
132651,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,Chinese engineer,member of the Chinese Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death
95037,26,Sergei Mavrodi,", 62, Russian financial fraudster, member of State Duma , heart attack.",https://en.wikipedia.org/wiki/Sergei_Mavrodi,33,2018,March,(1994–1995),None,Russian financial fraudster,member of State Duma,heart attack,None,None,None,None,None,None,None,62.0,None
22726,31,Fermín Vélez,", 43, Spanish sports car racing driver, two-time 12 Hours of Sebring winner, two-time Group C2 champion.",https://en.wikipedia.org/wiki/Ferm%C3%ADn_V%C3%A9lez,2,2003,March,None,None,Spanish sports car racing driver,two-time 12 Hours of Sebring winner,two-time Group C2 champion,None,None,None,None,None,None,None,43.0,None
54323,5,Bernard Rapoport,", 94, American businessman and philanthropist.",https://en.wikipedia.org/wiki/Bernard_Rapoport,8,2012,April,None,None,American businessman and philanthropist,None,None,None,None,None,None,None,None,None,94.0,None
66653,15,Dénes Zsigmondy,", 91, Hungarian classical violinist and music educator.",https://en.wikipedia.org/wiki/D%C3%A9nes_Zsigmondy,5,2014,February,None,None,Hungarian classical violinist and music educator,None,None,None,None,None,None,None,None,None,91.0,None
61933,1,Ian P. Howard,", 85, Canadian psychologist, cancer.",https://en.wikipedia.org/wiki/Ian_P._Howard,9,2013,June,None,None,Canadian psychologist,cancer,None,None,None,None,None,None,None,None,85.0,None


<IPython.core.display.Javascript object>

### Checking Data Types and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132652 entries, 0 to 132651
Data columns (total 21 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   day             132652 non-null  object 
 1   name            132652 non-null  object 
 2   info            132652 non-null  object 
 3   link            132652 non-null  object 
 4   num_references  132652 non-null  object 
 5   year            132652 non-null  int64  
 6   month           132652 non-null  object 
 7   info_parenth    49830 non-null   object 
 8   info_1          35 non-null      object 
 9   info_2          132604 non-null  object 
 10  info_3          62571 non-null   object 
 11  info_4          12605 non-null   object 
 12  info_5          1497 non-null    object 
 13  info_6          216 non-null     object 
 14  info_7          31 non-null      object 
 15  info_8          6 non-null       object 
 16  info_9          1 non-null       object 
 17  info_10   

<IPython.core.display.Javascript object>

#### Loading `nation_country_dict` from Pickle File to Dictionary `nation_map`

In [6]:
# Load the nation_country_dict
with open("nation_country_dict.pkl", "rb") as f:
    nation_map = pickle.load(f)

<IPython.core.display.Javascript object>

#### Function to Save Indices of Rows Matching Regular Expressions Pattern to a List and Print Number of Rows with Match 

In [7]:
# Define a function that takes dataframe, column name, and re pattern as arguments and returns list of indices
# for which column value matches re pattern
def rows_with_pattern(dataframe, column, pattern):
    """
    Takes input of dataframe, column name, and re pattern 
    and returns list of indices for rows that contain match
    for pattern anywhere within value for given column.
    
    dataframe: dataframe
    column: column name
    pattern: re pattern
    """
    index_list = []

    for i in dataframe.index:
        item = dataframe.loc[i, column]
        match = re.search(pattern, item)
        if match:
            index_list.append(i)
    print(
        f"There are {len(index_list)} rows with matching pattern in column '{column}'."
    )
    return index_list

<IPython.core.display.Javascript object>

#### Function to Use rows_with_pattern Function for Multiple Regular Expression Patterns

In [8]:
# Define a function that calls rows_with_pattern function for multiple re patterns
# returning a single list of indices for all rows with any pattern match


def multiple_patterns(dataframe, column, patterns):
    """
    Takes input dataframe, column, and list of re patterns and returns single list 
    of indices for rows in which a match for any pattern is found with re.search
    
    dataframe: dataframe
    column: column name
    patterns: list of re patterns
    """
    rows_combined = []

    # For loop to check each pattern
    for pattern in patterns:

        # List and number of rows matching each pattern
        print(pattern)
        rows_to_check = rows_with_pattern(dataframe, column, pattern)
        print("")

        # Add list for each pattern to combined list
        rows_combined += rows_to_check

    return rows_combined

<IPython.core.display.Javascript object>

## Extracting Nationality Continued
Here is the approach we will take:
- The plan will be to save the country name, in lieu of nationality, in new `place_1` and `place_2` columns as it is standardized for the various associated nationality values.
- First, we will update the keys and values in `nation_map` by replacing hyphens with a single space.
- Then we will remove "-born" from the column we are searching, as well as replace "-" and "/" each with single spaces.  In this step, we can also remove leading and trailing periods and whitespace.
- We will proceed to search the numbered `info_` columns in order, checking as follows:
    1. if column value starts with a value in the dictionary:
        - save country to `place_1` and remove value from searched column.
    2. if `place_1` value has been found:
        - if updated column value starts with a value in the dictionary:
            - save country to `place_2` and remove value from searched column.
    3. Repeat steps 1 and 2 but comparing with country (dictionary keys)
    4. Check unique values for column starting with capital letters.
- It is tempting to shorten the process by simply searching for nationality and country values within the column value, but as there are entries containing first and second nationality, we have to proceed from left to right to extract these values optimally.
- To avoid extracting incorrect values for a person who studies a place, rather than being from that place (e.g., "Egypt" vs "Egyptologist"), we will search for the exact value and the value with a single trailing space.

In [9]:
nation_map

{'Afghan': 'Afghanistan',
 'Albanian': 'Albania',
 'Algerian': 'Algeria',
 'Andorran': 'Andorra',
 'Angolan': 'Angola',
 'Antiguan': 'Antigua and Barbuda',
 'Barbudan': 'Antigua and Barbuda',
 'Argentine': 'Argentina',
 'Armenian': 'Armenia',
 'Australian': 'Australia',
 'Austrian': 'Austria',
 'Azerbaijani': 'Azerbaijan',
 'Azeri': 'Azerbaijan',
 'Bahamian': 'The Bahamas',
 'Bahraini': 'Bahrain',
 'Bengali': 'Bangladesh',
 'Barbadian': 'Barbados',
 'Belarusian': 'Belarus',
 'Belgian': 'Belgium',
 'Belizean': 'Belize',
 'Beninese': 'Benin',
 'Beninois': 'Benin',
 'Bhutanese': 'Bhutan',
 'Bolivian': 'Bolivia',
 'Bosnian': 'Bosnia and Herzegovina',
 'Herzegovinian': 'Bosnia and Herzegovina',
 'Motswana': 'Botswana',
 'Botswanan': 'Botswana',
 'Brazilian': 'Brazil',
 'Bruneian': 'Brunei',
 'Bulgarian': 'Bulgaria',
 'Burkinabé': 'Burkina Faso',
 'Burmese': 'Burma',
 'Burundian': 'Burundi',
 'Cabo Verdean': 'Cabo Verde',
 'Cambodian': 'Cambodia',
 'Cameroonian': 'Cameroon',
 'Canadian': 'Ca

<IPython.core.display.Javascript object>

#### Removing "-" and "." from `nation_map`

In [10]:
# Removing hyphens from nation_map
nation_map = {
    key.replace("-", ""): value.replace("-", " ") for key, value in nation_map.items()
}

# Removing periods from nation_map
nation_map = {key.replace(".", ""): value for key, value in nation_map.items()}

<IPython.core.display.Javascript object>

#### Removing or Replacing Extra Characters in `info_` Columns

In [11]:
%%time

# List of columns to treat
cols_lst = [
    "info_1",
    "info_2",
    "info_3",
    "info_4",
    "info_5",
    "info_6",
    "info_7",
    "info_8",
    "info_9",
    "info_10",
    "info_11",
    'info_parenth'
]

# Dictionary of keys to find and values to replace keys
replace_dict = {'-born': '', '–born': '', '-bred': '', '–bred': '', '-': ' ', '–': ' ', '/': ' ', '.': ' ', '(': '', ')': '', "'s": '', "   ": ' ', '  ': ' '}

# For loop to find and replace characters in replace_dict in columns in cols_list
# and strip any leading or trailing periods or whitespace
for column in cols_lst:
    for key, value in replace_dict.items():
        for index in df[column].notna().index:
            item = df.loc[index, column]
            if item:
                df.loc[index, column] = item.replace(key, value).strip()

CPU times: total: 4min 42s
Wall time: 4min 42s


<IPython.core.display.Javascript object>

#### Initializing `place_1` and `place_2` Columns

In [12]:
# Initializing place_1 and place_2
df[["place_1", "place_2"]] = None

# Checking a sample
df.sample()

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
99504,22,Len Campbell,", 71, Scottish footballer .",https://en.wikipedia.org/wiki/Len_Campbell,3,2018,November,Dumbarton,None,Scottish footballer,None,None,None,None,None,None,None,None,None,71.0,None,None,None


<IPython.core.display.Javascript object>

#### Checking `info_1` for `place_1`

In [13]:
# Column to check
column = "info_1"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[column].notna()) & df[extract_to].isna()]
    for index in dataframe.index:
        item = df.loc[index, column]
        if (
            item == nationality
            or item == country
            or item.startswith(nationality + " ")
            or item.startswith(country + " ")
        ):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column]
                .replace(nationality, "")
                .strip()
                .replace(country, "")
                .strip()
            )

# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
19580,20,Dame Miraka Szászy,", 80. New Zealand Maori leader.",https://en.wikipedia.org/wiki/Mira_Sz%C3%A1szy,21,2001,December,None,Maori leader,None,None,None,None,None,None,None,None,None,None,80.0,None,New Zealand,None
61945,1,Basil Soper,", British actor, 74–75.",https://en.wikipedia.org/wiki/Basil_Soper,0,2013,June,None,actor,None,None,None,None,None,None,None,None,None,None,74.5,None,United Kingdom of Great Britain and Northern Ireland,None


<IPython.core.display.Javascript object>

#### Observations:
- `info_1` provides us a small sample on which to test code.
- We successfully extracted those `place_1` values, now we will do the same on the treated rows for `place_2`.

#### Checking `info_1` for `place_2`

In [14]:
# Column to check
column = "info_1"

# Extract to column
extract_to = "place_2"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df["place_1"].notna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if (
            item == nationality
            or item == country
            or item.startswith(nationality + " ")
            or item.startswith(country + " ")
        ):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column]
                .replace(nationality, "")
                .strip()
                .replace(country, "")
                .strip()
            )

# Check a sample of rows
df.sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
28855,11,Murugappa Channaveerappa Modi,", 89, Indian ophthalmologist.",https://en.wikipedia.org/wiki/Murugappa_Channaveerappa_Modi,8,2005,November,None,None,Indian ophthalmologist,None,None,None,None,None,None,None,None,None,89.0,None,None,None
44680,22,Gene Lees,", 82, Canadian jazz historian and critic, heart disease.",https://en.wikipedia.org/wiki/Gene_Lees,5,2010,April,None,None,Canadian jazz historian and critic,heart disease,None,None,None,None,None,None,None,None,82.0,None,None,None


<IPython.core.display.Javascript object>

#### Observations:
- Let us check the remaining unique values in `info_1`.

#### Checking Remaining Unique Values in `info_1`

In [15]:
# Checking unique values
df["info_1"].unique()

array([None, 'politician', 'Olympic sprinter', 'gridiron football player',
       'writer', 'businessman', 'social psychologist', 'King of Nepal',
       'Maori leader', 'artist', 'English sports journalist',
       'Jules Engel', 'early', 'aka', 'Jr', 'professional wrestler',
       'automotive engineer', 'materials scientist', 'weightlifter',
       'common chimpanzee', '', 'Olympic athlete', 'actor',
       'Olympic gymnast', 'broadcaster and writer', 'Olympic swimmer',
       'Olympic boxer', 'Olympic wrestler', 'Olympic sailor',
       'basketball player', 'college basketball coach',
       'choral conductor', 'Tree of the Year'], dtype=object)

<IPython.core.display.Javascript object>

#### Obsservations:
- Neither "English" nor "Maori" are keys in the current dictionary.
- Maori is an ethnicity within the country of New Zealand, so for now, we will add it as a key our dictionary with the country value of New Zealand.  If we have matching first and second countries, we can later remove the duplicate value.
- We will also add the key "English" with the country value 'United Kingdom of Great Britain and Northern Ireland'.
- Then, we can rerun the above code for `place_1` and `place_2`.

#### Updating `nation_map`

In [16]:
# Adding key: country pairs to nation_map
nation_map["English"] = nation_map["British"]
nation_map["Maori"] = nation_map["New Zealand"]

<IPython.core.display.Javascript object>

#### Re-checking `info_1` for `place_1`

In [17]:
# Column to check
column = "info_1"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if (
            item == nationality
            or item == country
            or item.startswith(nationality + " ")
            or item.startswith(country + " ")
        ):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column]
                .replace(nationality, "")
                .strip()
                .replace(country, "")
                .strip()
            )

# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
8861,13,George Strugar,", 63. American gridiron football player, lung cancer.",https://en.wikipedia.org/wiki/George_Strugar,0,1997,June,None,gridiron football player,lung cancer,None,None,None,None,None,None,None,None,None,63.0,None,United States of America,None
12317,25,Sir Robin Brook,", 90 British businessman, banker and Olympic fencer.",https://en.wikipedia.org/wiki/Robin_Brook,3,1998,October,None,businessman,banker and Olympic fencer,None,None,None,None,None,None,None,None,None,90.0,None,United Kingdom of Great Britain and Northern Ireland,None


<IPython.core.display.Javascript object>

#### Re-checking `info_1` for `place_2`

In [18]:
# Column to check
column = "info_1"

# Extract to column
extract_to = "place_2"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[
        (df["place_1"].notna()) & (df[extract_to].isna()) & (df[column].notna())
    ]
    for index in dataframe.index:
        item = df.loc[index, column]
        if (
            item == nationality
            or item == country
            or item.startswith(nationality + " ")
            or item.startswith(country + " ")
        ):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column]
                .replace(nationality, "")
                .strip()
                .replace(country, "")
                .strip()
            )

# Checking rows
df[df["place_2"].notna()]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
17841,28,Marie Jahoda,", 94 Austrian-British social psychologist.",https://en.wikipedia.org/wiki/Marie_Jahoda,3,2001,April,None,social psychologist,None,None,None,None,None,None,None,None,None,None,94.0,None,Austria,United Kingdom of Great Britain and Northern Ireland
19580,20,Dame Miraka Szászy,", 80. New Zealand Maori leader.",https://en.wikipedia.org/wiki/Mira_Sz%C3%A1szy,21,2001,December,None,leader,None,None,None,None,None,None,None,None,None,None,80.0,None,New Zealand,New Zealand


<IPython.core.display.Javascript object>

#### Observations:
- Our code appears to be finding the matching values and assigning the corresponding country to the correct nation column.
- We see "New Zealand" added to both nation columns here, which was expected as both New Zealand and Maori are in the description
- As an aside, we will need to check our final values where `place_1` is "American" and `place_2` is "Indian" as our code will indicate United States and India, which may or may not be correct. 
- Now we can proceed to doing the same extraction on `info_2`.

#### Checking `info_2` for `place_1`

In [19]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item == nationality or item == country or item.startswith(nationality + ' ') or item.startswith(country + ' '):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            )

CPU times: total: 2min 51s
Wall time: 2min 51s


<IPython.core.display.Javascript object>

In [20]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
49663,24,Barry Potomski,", 38, Canadian ice hockey player .",https://en.wikipedia.org/wiki/Barry_Potomski,1,2011,May,Los Angeles Kings,None,ice hockey player,None,None,None,None,None,None,None,None,None,38.0,None,Canada,None
116885,2,Jean René Allard,", 90, Canadian politician, member of the Manitoba Legislature .",https://en.wikipedia.org/wiki/Jean_Ren%C3%A9_Allard,5,2020,December,1969 1973,None,politician,member of the Manitoba Legislature,None,None,None,None,None,None,None,None,90.0,None,Canada,None


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for place_1

In [21]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 2465 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Observations:
- We have captured the `place_1` value for the vast majority of entries.
- Next we will check for other variations on nationality usage.

#### Examining Unique Values of First Word in `info_1` if Upper Case

In [22]:
# Column to check
column = "info_2"

# Dataframe to check
dataframe = df[(df["place_1"].isna()) & (df[column].notna())]

# Checking set of first words in info_2 where place_1 is missing
first_words = set([item.split()[0] for item in dataframe[column] if item[0].isupper()])
print(
    f"There are {len(first_words)} unique values starting with upper case for first word in info_1.\n"
)
first_words

There are 324 unique values starting with upper case for first word in info_1.



{'AIDS',
 'ANC',
 'Abkhaz',
 'Abkhazian',
 'Aboriginal',
 'Actress',
 'Afghani',
 'African',
 'Afrikaans',
 'Afrikaner',
 'Afro',
 'Air',
 'Alfa',
 'All',
 'Alyawarre',
 'Amateur',
 'America',
 'American]]',
 'Americane',
 'Americanthoroughbred',
 'Amrican',
 'Anglican',
 'Anglo',
 'Anguillan',
 'Arabic',
 'Archbishop',
 'Archdeacon',
 'Argentinian',
 'Aruba',
 'Aruban',
 'Assamese',
 'Associate',
 'Assyrian',
 'Athletics',
 'Aussie',
 'Australiancabaret',
 'Austro',
 'Avarian',
 'Azorean',
 'BBC',
 'Baltic',
 'Basque',
 'Bavarian',
 'Belarussian',
 'Benedictine',
 'Bermudan',
 'Bermudian',
 'Bessarabian',
 'Bletchley',
 'Bodo',
 'Bosnia',
 'Braziliam',
 'Breton',
 'Brigadier',
 'Britain',
 'Britsih',
 'Bulgarianactor',
 'California',
 'Californian',
 'Calypso',
 'Cantonese',
 'Caribbean',
 'Catalan',
 'Catholic',
 'Caymanian',
 'Ceylon',
 'Ceylonese',
 'Chagossian',
 'Chairman',
 'Chechen',
 'Cherokee',
 'Chief',
 'Chilen',
 'Chileán',
 'Chilian',
 'China',
 'Chiricahua',
 'Chuvash',


<IPython.core.display.Javascript object>

#### Observations:
- We can see there are some remaining variations on how nationality was entered that are not yet in `nation_country_dict`.
- Let us add those now, then do another iteration searching `info_2`.
- Descriptions will be assigned to their geographical physical region where sovereign state is remote or nationality description is broad.

#### Hard-coding Additional Variations on Nationality

In [23]:
# Hard-coding remaining unique nationality/location descriptors
nation_map["ANC"] = nation_map["South African"]
nation_map["Abkhaz"] = nation_map["Georgian"]
nation_map["Abkhazian"] = nation_map["Georgian"]
nation_map["Aboriginal"] = nation_map["Australian"]
nation_map["African"] = "Africa"
nation_map["Afrikaans"] = nation_map["African"]
nation_map["Afrikaner"] = nation_map["African"]
nation_map["Afro"] = "Africa"
nation_map["Alyawarre"] = nation_map["Australian"]
nation_map["America"] = nation_map["US"]
nation_map["America's"] = nation_map["US"]
nation_map["Amrican"] = nation_map["US"]
nation_map["Anglo"] = "Europe"
nation_map["Anguillan"] = "Central America and the Caribbean"
nation_map["Antigua"] = nation_map["Antiguan"]
nation_map["Arabic"] = "Arab states"
nation_map["Argentinian"] = nation_map["Argentine"]
nation_map["Aruba"] = "Central America and the Caribbean"
nation_map["Aruban"] = nation_map["Aruba"]
nation_map["Assamese"] = nation_map["Indian"]
nation_map["Assyrian"] = "Middle East"
nation_map["Aussie"] = nation_map["Australian"]
nation_map["Australia"] = nation_map["Australian"]
nation_map["Australia's"] = nation_map["Australian"]
nation_map["Austria"] = nation_map["Austrian"]
nation_map["Austro"] = nation_map["Austrian"]
nation_map["Avarian"] = nation_map["Russian"]
nation_map["Azerbaijan"] = nation_map["Azerbaijani"]
nation_map["Azorean"] = nation_map["Portuguese"]
nation_map["Baltic"] = "Europe"
nation_map["Bangladesh"] = nation_map["Bangladeshi"]
nation_map["Barbados"] = "Central America and the Caribbean"
nation_map["Basque"] = "Europe"
nation_map["Bavarian"] = nation_map["German"]
nation_map["Belarus"] = nation_map["Belarusian"]
nation_map["Belarussian"] = nation_map["Belarusian"]
nation_map["Belgium"] = nation_map["Belgian"]
nation_map["Bermudan"] = "Central America and the Caribbean"
nation_map["Bermudian"] = "Central America and the Caribbean"
nation_map["Bessarabian"] = "Europe"
nation_map["Bletchley"] = nation_map["British"]
nation_map["Bodo"] = nation_map["Norwegian"]
nation_map["Bosnia"] = nation_map["Bosnian"]
nation_map["Breton"] = nation_map["French"]
nation_map["Britain's"] = nation_map["British"]
nation_map["Britsih"] = nation_map["British"]
nation_map["California"] = nation_map["US"]
nation_map["Californian"] = nation_map["US"]
nation_map["Cantonese"] = nation_map["Chinese"]
nation_map["Caribbean"] = "Central America and the Caribbean"
nation_map["Catalan"] = nation_map["Spanish"]
nation_map["Caymanian"] = nation_map["Caribbean"]
nation_map["Ceylon"] = nation_map["Sri Lankan"]
nation_map["Ceylonese"] = nation_map["Sri Lankan"]
nation_map["Chagossian"] = "Indian Ocean"
nation_map["Chechen"] = nation_map["Russian"]
nation_map["Cherokee"] = nation_map["US"]
nation_map["Chilian"] = nation_map["Chilean"]
nation_map["China"] = nation_map["Chinese"]
nation_map["Chiricahua"] = nation_map["US"]
nation_map["Chuvash"] = nation_map["Russian"]
nation_map["Circassian"] = nation_map["Russian"]
nation_map["Columbian"] = nation_map["Colombian"]
nation_map["Congo"] = nation_map["Congolese"]
nation_map["Congoleze"] = nation_map["Congolese"]
nation_map["Cornish"] = nation_map["British"]
nation_map["Costan Rican"] = nation_map["Costa Rican"]
nation_map["Côte d'Ivoire"] = nation_map["Ivorian"]
nation_map["Crimean"] = nation_map["Russian"]
nation_map["Croat"] = nation_map["Croatian"]
nation_map["Curaçaoan"] = nation_map["Dutch"]
nation_map["Curaçaon"] = nation_map["Dutch"]
nation_map["Dagestani"] = nation_map["Russian"]
nation_map["Dahomey"] = "Africa"
nation_map["Dijiboutian"] = nation_map["Djiboutian"]
nation_map["Dolgan"] = nation_map["Russian"]
nation_map["Dominica"] = nation_map["Caribbean"]
nation_map["England"] = nation_map["British"]
nation_map["Englist"] = nation_map["British"]
nation_map["European"] = "Europe"
nation_map["Falkland Islands"] = "South America"
nation_map["Falkland islands"] = nation_map["Falkland Islands"]
nation_map["Falkland"] = nation_map["Falkland Islands"]
nation_map["Faroese"] = nation_map["Danish"]
nation_map["Filipina"] = nation_map["Filipino"]
nation_map["Filipo"] = nation_map["Filipino"]  # verified entry
nation_map["Fillipina"] = nation_map["Filipino"]
nation_map["Finish"] = nation_map["Finnish"]
nation_map["Flemish"] = nation_map["Belgian"]
nation_map["Franch"] = nation_map["French"]  # verified entry
nation_map["Franco"] = nation_map["French"]
nation_map["Frenck"] = nation_map["French"]  # verified entry
nation_map["Fujianese"] = nation_map["Chinese"]
nation_map[
    "Gaelic"
] = "Europe"  # refers to sport of Gaelic football, otherwise language
nation_map["Galician"] = nation_map["Spanish"]
nation_map["Galápagos"] = nation_map["Ecuadorian"]  # entry for tortoise
nation_map["Geman"] = nation_map["German"]  # verified entry
nation_map["Germen"] = nation_map["German"]  # verified entry
nation_map["Ghanese"] = "Africa"
nation_map["Greenlandic"] = nation_map["Danish"]
nation_map["Guadeloupean"] = nation_map["Caribbean"]
nation_map["Guamanian"] = "Oceania"
nation_map["Guernsey"] = nation_map["British"]
nation_map["Guernseyan"] = nation_map["British"]
nation_map["Hawaiian"] = nation_map["US"]
nation_map["Hollywood"] = nation_map["US"]
nation_map["Hong Kong"] = nation_map["Chinese"]
nation_map["Houston"] = nation_map["US"]
nation_map["Huaorani"] = nation_map["Ecuadorian"]
nation_map["I Kiribati"] = nation_map["IKiribati"]
nation_map["Indin"] = nation_map["Indian"]  # verified entry
nation_map["Indo"] = nation_map["Indian"]
nation_map["Ingush"] = nation_map["Russian"]
nation_map["Italo"] = nation_map["Italian"]
nation_map["Ivoirian"] = "Africa"
nation_map["Javanese"] = nation_map["Indonesian"]
nation_map["Jersey"] = nation_map["British"]
nation_map["Kabardin"] = nation_map["Russian"]
nation_map["Kashmiri"] = nation_map["Indian"]
nation_map["Korean"] = "Asia"
nation_map["Kosovan"] = "Europe"
nation_map["Kosovar"] = "Europe"
nation_map["Kosovo"] = "Europe"
nation_map["Kurdish"] = "Asia"
nation_map["Lesothan"] = "Africa"
nation_map["Los Angeles"] = nation_map["US"]
nation_map["Louisiana"] = nation_map["US"]
nation_map["MGerman"] = nation_map["German"]  # verified entry
nation_map["Macanese"] = nation_map["Chinese"]
nation_map["Malayalam"] = nation_map["Indian"]
nation_map["Malayali"] = nation_map["Indian"]
nation_map["Malayan"] = nation_map["Malaysian"]
nation_map["Manx"] = nation_map["British"]
nation_map["Mexian"] = nation_map["Mexican"]
nation_map["Mississippi"] = nation_map["US"]
nation_map["Monegasque"] = nation_map["Monacan"]
nation_map["Montserrat"] = nation_map["Caribbean"]
nation_map["Montserratian"] = nation_map["Caribbean"]
nation_map["Myanmar"] = nation_map["Burmese"]
nation_map["New York"] = nation_map["US"]
nation_map["Ngarrindjeri"] = nation_map["Australian"]
nation_map["Ni Vanuatu"] = "Oceania"
nation_map["Nigirean"] = nation_map["Nigerian"]
nation_map["Niuean"] = nation_map["NZ"]
nation_map["Northern Ire"] = nation_map["Northern Irish"]
nation_map["Northern Ireland"] = nation_map["Northern Irish"]
nation_map["Norther Irish"] = nation_map["Northern Irish"]
nation_map["North Irish"] = nation_map["Northern Irish"]
nation_map["North American"] = "North America"
nation_map["North Island"] = nation_map["NZ"]
nation_map["Northern Mariana Island"] = "Oceania"
nation_map["Northern Mariana Islander"] = "Oceania"
nation_map["Nubian"] = nation_map["Sudanese"]
nation_map["Ottoman"] = nation_map["Turkish"]
nation_map["Paraguan"] = nation_map["Paraguayan"]  # verified entry
nation_map["People's Republic of China"] = nation_map["Chinese"]
nation_map["Pitcairn"] = "Oceania"
nation_map["Poliosh"] = nation_map["Polish"]  # verified entry
nation_map["Polis"] = nation_map["Polish"]  # verified entry
nation_map["Prussian"] = nation_map["German"]
nation_map["Punjabi"] = nation_map["Indian"]
nation_map["Quebec"] = nation_map["Canadian"]
nation_map["Québécois"] = nation_map["Canadian"]
nation_map["Republic of China"] = nation_map["Chinese"]
nation_map["Rhodesian"] = "Africa"
nation_map["Roman"] = nation_map["Italian"]
nation_map["Réunionese"] = nation_map["French"]
nation_map["S African"] = nation_map["South African"]
nation_map["Saban"] = nation_map["Caribbean"]
nation_map["Saharawi"] = "Africa"
nation_map["Sahrawi"] = nation_map["Saharawi"]
nation_map["Saint Helena"] = "South Atlantic"
nation_map["Saint Vincent"] = nation_map["Caribbean"]
nation_map["Saint Martin"] = nation_map["Caribbean"]
nation_map["Saint Pierre and Miquelon"] = "North America"
nation_map["Salvadorean"] = nation_map["Salvadoran"]
nation_map["Sanmarinese"] = nation_map["Sammarinese"]
nation_map["Santomean"] = nation_map["São Toméan"]
nation_map["Seychellian"] = nation_map["Seychellois"]
nation_map["Sicilian"] = nation_map["Italian"]
nation_map["Sicillian"] = nation_map["Italian"]
nation_map["Sikkimese"] = nation_map["Indian"]
nation_map["Sorbian"] = nation_map["German"]
nation_map["South Afican"] = nation_map["South African"]
nation_map["South Ossetian"] = nation_map["Georgian"]
nation_map["Soviet"] = "United Socialist Soviet Republic"
nation_map["Sri lankan"] = nation_map["Sri Lankan"]
nation_map["St Lucian"] = nation_map["Caribbean"]
nation_map["St Kitts and Nevis"] = nation_map["Kittian and Nevisian"]
nation_map["Sumatran"] = nation_map["Indonesian"]
nation_map["Swish"] = nation_map["Swiss"]  # verified entry
nation_map["Tahitian"] = "South Pacific"
nation_map["Tamil"] = "Indian subcontinent"
nation_map["Tasmanian"] = nation_map["Australian"]
nation_map["Telugu"] = nation_map["Indian"]
nation_map["Texas"] = nation_map["US"]
nation_map["Florida"] = nation_map["US"]
nation_map["Tibetan"] = nation_map["Chinese"]
nation_map["Tirkish"] = nation_map["Turkish"]
nation_map["Tirkish"] = nation_map["Turkish"]
nation_map["Trinbagonian"] = nation_map["Trinidadian"]
nation_map["Trinidad"] = nation_map["Trinidadian"]
nation_map["Turks"] = nation_map["Turkish"]
nation_map["U S"] = nation_map["US"]
nation_map["UAE's"] = nation_map["Emirati"]
nation_map["United Kingdom"] = nation_map["British"]
nation_map["Upper Silesian"] = nation_map["Polish"]
nation_map["Uruguyan"] = nation_map["Uruguayan"]
nation_map["Uyghur"] = "Asia"
nation_map["Ni Vanuatu"] = nation_map["Vanuatuan"]
nation_map["Wallisian"] = "Oceania"
nation_map["Xhosa"] = nation_map["South African"]
nation_map["Yazidi"] = "Asia"
nation_map["Yellowstone"] = nation_map["US"]
nation_map["Yugoslav"] = nation_map["Serbian"]
nation_map["Yugoslavia"] = nation_map["Serbian"]
nation_map["Yugoslavian"] = nation_map["Serbian"]
nation_map["Zairean"] = nation_map["Congolese"]
nation_map["Zanzibari"] = nation_map["Tanzanian"]
nation_map["Czechoslovakian"] = nation_map[
    "Czech"
]  # Note:  this will later be converted to Europe, so either Czech or Slovak works here
nation_map["Czechoslovak"] = nation_map["Czech"]  # same as above

<IPython.core.display.Javascript object>

#### Example of Checking Rows with Unique Value

In [24]:
# Cell for checking rows of unique values -- used repeatedly while hard-coding above
check_value = "Yellowstone "

check_list = []
for index in df[df["info_2"].notna()].index:
    item = df.loc[index, "info_2"]
    if item:
        if item.startswith(check_value):
            check_list.append(index)
df.loc[check_list, :]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
58424,6,O-Six,", 6, Yellowstone National Park gray wolf, shot.",https://en.wikipedia.org/wiki/O-Six,16,2012,December,None,None,Yellowstone National Park gray wolf,shot,None,None,None,None,None,None,None,None,6.0,None,None,None


<IPython.core.display.Javascript object>

#### Observations:
- Because the original `info` column contains mutliple pieces of information, in various formats, we will be doing some hard-coding along the way.
- We will proceed to re-run our code searching `info_2`.

#### Checking `info_2` for `place_1` with Updated `nation_map`

In [25]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_1"


# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item==nationality or item==country or item.startswith(nationality + ' ') or item.startswith(country + ' '):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            )

CPU times: total: 10.4 s
Wall time: 10.4 s


<IPython.core.display.Javascript object>

In [26]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
18867,16,Max Ephraim Jr,", 82, American railroad mechanical engineer, aided transition from steam-powered to diesel-electric locomotives.",https://en.wikipedia.org/wiki/Max_Ephraim_Jr,0,2001,September,None,None,railroad mechanical engineer,aided transition from steam powered to diesel electric locomotives,None,None,None,None,None,None,None,None,82.0,None,United States of America,None
127070,6,Clifford Rose,", 92, British actor .",https://en.wikipedia.org/wiki/Clifford_Rose,9,2021,November,", ,",None,actor,None,None,None,None,None,None,None,None,None,92.0,None,United Kingdom of Great Britain and Northern Ireland,None


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [27]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 750 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Obervations:
- We have narrowed down the missing values for `place_1` values to ~750.
- Let us take another look at the remaining unique values for `info_2`.

#### Checking Remaining Unique Values for `info_2` Where `place_1` Value is Missing

In [28]:
# Checking remaining unique values for info_2 where place_1 value is missing
df[df["place_1"].isna()]["info_2"].unique()

array(['Royal Netherlands Navy vice admiral', 'President of Laos',
       'Luxembourgian road bicycle racer',
       'Governor general of the Bahamas',
       'Amateur violinist and philanthropist',
       'West German long distance runner and Olympian',
       'Composer and music editor',
       'President of the Yemen Arab Republic',
       'Luxembourgian football player', 'Native American tribal leader',
       'Prime Minister of Rwanda',
       'Male Hungarian international table tennis player',
       '37th President of the United States',
       'Queen of Jordan as the wife of King Talal',
       'Prime Minister of Zaire under Mobutu Sese Seko',
       'President of Burma and writer',
       'Founder and first leader of North Korea',
       'Irani American engineer', 'Moravian American classical pianist',
       'President of Ciskei', 'Prime Minister of Nepal',
       'President of Palau', 'Poet and an Esperantist professor',
       'East German politician', 'People Republic of C

<IPython.core.display.Javascript object>

#### Observations:
- We can see some country names embedded in the middle of the values, so we will search there next, for nationality or country.
- The first place value found will be maintained, which does bias the value toward nationalities/countries higher up on the list, as we are not proceeding from left to right for this iteration, but we anticipate the impact to be trivial.

#### Checking `info_2` for `place_1` via Nationality or Country within Value

In [29]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if nationality + ' ' in item or country + ' ' in item or item.endswith(nationality) or item.endswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            )

CPU times: total: 5.67 s
Wall time: 5.68 s


<IPython.core.display.Javascript object>

In [30]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
78300,28,Jorge Scarso,", 99, Italian-born Brazilian Roman Catholic prelate, Bishop of Patos de Minas .",https://en.wikipedia.org/wiki/Jorge_Scarso,0,2015,October,1967 1992,None,Brazilian Roman Catholic prelate,Bishop of Patos de Minas,None,None,None,None,None,None,None,None,99.0,None,Italy,None
101546,23,Roger Ainsworth,", 67, British engineer, cancer.",https://en.wikipedia.org/wiki/Roger_Ainsworth,11,2019,February,None,None,engineer,cancer,None,None,None,None,None,None,None,None,67.0,None,United Kingdom of Great Britain and Northern Ireland,None


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [31]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 390 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Observations:
- We have ~400 remaining missing values for `place_1`.
- After checking these unique `info_2` values for these rows, we can update the dictionary again.

In [32]:
# Checking remaining unique values for info_2 where place_1 value is missing
df[df["place_1"].isna()]["info_2"].unique()

array(['Luxembourgian road bicycle racer',
       'Governor general of the Bahamas',
       'Amateur violinist and philanthropist',
       'Composer and music editor', 'Luxembourgian football player',
       'Prime Minister of Zaire under Mobutu Sese Seko',
       'President of Ciskei', 'Poet and an Esperantist professor',
       'Jewish rabbi', 'Iranist',
       'Somalian military leader and statesman',
       "native leader and historian of the Metepenagiag Mi'kmaq Nation",
       'Luxembourgian cyclist',
       'sidecarcross rider and the first ever Sidecarcross World Championship',
       'Major League Baseball player', 'professional wrestler',
       'Field hockey player', 'fifth dean of the Harvard Business School',
       'Racecar driver', 'longtime chairman of Liverpool F C',
       'Iraniab historian', 'Orthodox Jewish rabbi',
       'Royal Air Force Air marshal and flying ace during World War II',
       'Archdeacon of Halifax',
       'inspector general and executive directo

<IPython.core.display.Javascript object>

#### Hard-coding Additional Variations on Nationality

In [33]:
# Hard-coding remaining unique nationality/location descriptors
nation_map["Bahamas"] = nation_map["Bahamian"]
nation_map["Zaire"] = nation_map["Zairean"]
nation_map["Ciskei"] = "Africa"
nation_map["Metepenagiag Mi'kmaq"] = nation_map["Canadian"]
nation_map["Major League"] = nation_map["US"]
nation_map["Liverpool"] = nation_map["British"]
nation_map["Halifax"] = nation_map["Canadian"]
nation_map["Major Leagues"] = nation_map["US"]
nation_map["Levi Strauss & Co"] = nation_map["US"]
nation_map["Miami"] = nation_map["US"]
nation_map["Heaven Gate"] = nation_map["US"]
nation_map["Royal Navy"] = nation_map["British"]
nation_map["Luftwaffe"] = nation_map["German"]
nation_map["Ashanti"] = "Africa"
nation_map["White House"] = nation_map["US"]
nation_map["Boeing"] = nation_map["US"]
nation_map["Red Army"] = nation_map["Soviet"]
nation_map["Yokohama"] = nation_map["Japanese"]
nation_map["Tasmania"] = nation_map["Tasmanian"]
nation_map["Harvard"] = nation_map["US"]
nation_map["Malaya"] = nation_map["Malayan"]
nation_map["Pennsylvania"] = nation_map["US"]
nation_map["House of Saud"] = nation_map["Saudi"]
nation_map["Faroe Islands"] = nation_map["Faroese"]
nation_map["Kentucky"] = nation_map["US"]
nation_map["Norfolk"] = nation_map["British"]  # verified entry
nation_map["Rhodesia"] = nation_map["Rhodesian"]
nation_map["Royal Marines"] = nation_map["British"]
nation_map["Rhode Island"] = nation_map["US"]
nation_map["Jerusalem"] = nation_map["Israeli"]
nation_map["Kriegsmarine"] = nation_map["German"]
nation_map["Nazi"] = nation_map["German"]
nation_map["South Carolina"] = nation_map["US"]
nation_map["Rashtriya Swayamsevak Sangh"] = nation_map["Indian"]
nation_map["Ontario"] = nation_map["Canadian"]
nation_map["London"] = nation_map["British"]
nation_map["McDonalds"] = nation_map["US"]
nation_map["Kleiner"] = nation_map["US"]
nation_map["Worldcom"] = nation_map["US"]
nation_map["Reino Aventura"] = nation_map["Mexican"]
nation_map["Palm Beach"] = nation_map["US"]
nation_map["Lawrence Radiation Laboratory"] = nation_map["US"]
nation_map["Royal Air Force"] = nation_map["British"]
nation_map["Britain"] = nation_map["British"]
nation_map["american"] = nation_map["US"]
nation_map["St  Lucian"] = nation_map["St Lucian"]
nation_map["Cook Islands"] = "Oceania"
nation_map["E O  Green Junior High School"] = nation_map["US"]
nation_map["Cook Islander"] = "Oceania"
nation_map["western lowland"] = "Africa"
nation_map["Saharan"] = "Africa"
nation_map["St  Kitts and Nevis"] = nation_map["Kittian"]
nation_map["Jammu and Kashmir"] = "Indian subcontinent"
nation_map["Cook Island"] = "Oceania"
nation_map["New Caledonian"] = "Oceania"
nation_map["Luxembourgian"] = "Luxembourg"
nation_map["Iraniab"] = nation_map["Iranian"]
nation_map["Australiancabaret"] = nation_map["Australian"]
nation_map["Chileán"] = nation_map["Chilean"]
nation_map["Somalian"] = nation_map["Somali"]
nation_map["Americanthoroughbred"] = nation_map["US"]
nation_map["USAF"] = nation_map["US"]
nation_map["Egyption"] = nation_map["Egyptian"]
nation_map["USA"] = nation_map["US"]

<IPython.core.display.Javascript object>

#### Re-checking `info_2` for `place_1` via Nationality or Country within Value after Updating `nation_map`

In [34]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if nationality + ' ' in item or country + ' ' in item or item.endswith(nationality) or item.endswith(country):
            df.loc[index, extract_to] = country
            df.loc[index, column] = (
                df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
            )

CPU times: total: 6.86 s
Wall time: 6.85 s


<IPython.core.display.Javascript object>

In [35]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
102888,28,Caroline Bittencourt,", 37, Brazilian model, drowned.",https://en.wikipedia.org/wiki/Caroline_Bittencourt,9,2019,April,None,None,model,drowned,None,None,None,None,None,None,None,None,37.0,None,Brazil,None
46884,15,N. Paul Kenworthy,", 85, American cinematographer , thyroid cancer.",https://en.wikipedia.org/wiki/N._Paul_Kenworthy,1,2010,October,",",None,cinematographer,thyroid cancer,None,None,None,None,None,None,None,None,85.0,None,United States of America,None


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [36]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 229 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Observations:
- We have assigned some "nationality" keys, such as "Nazi", which has the value "German", which may vary from the specific nation (e.g., "Austria"), however, our plan is to later convert to geographical region, so these values serve our purpose.
- There are ~230 remaining missing values for `place_1`, which we do not expect to find in `info_2`.
- We can now move on to searching for `place_2` in `info_2`.

#### Checking `info_2` for `place_2`

In [37]:
%%time

# Column to check
column = "info_2"

# Extract to column
extract_to = "place_2"

# For loop to extract nation data to place column
for nationality, country in nation_map.items():
    dataframe = df[
    (df["place_1"].notna()) & (df[extract_to].isna()) & (df[column].notna())]
    for index in dataframe.index:
        item = df.loc[index, column]
        if item == nationality or item == country or item.startswith(nationality + ' ') or item.startswith(country + ' '):
            df.loc[index, extract_to] = country
            df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()

CPU times: total: 8min 56s
Wall time: 8min 56s


<IPython.core.display.Javascript object>

In [38]:
# Check a sample of rows
df[df[extract_to].notna()].sample(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
78833,29,Otto Newman,", 93, Austrian-born British sociologist.",https://en.wikipedia.org/wiki/Otto_Newman,4,2015,November,None,None,British sociologist,None,None,None,None,None,None,None,None,None,93.0,None,Austria,United Kingdom of Great Britain and Northern Ireland
39595,21,Victor Zarnowitz,", 89, Polish-born American economist, heart attack.",https://en.wikipedia.org/wiki/Victor_Zarnowitz,5,2009,February,None,None,American economist,heart attack,None,None,None,None,None,None,None,None,89.0,None,Poland,United States of America


<IPython.core.display.Javascript object>

#### Checking Number of Entries with Values for `place_2`

In [39]:
# Checking number of entries for place_2
print(f'There are {df["place_2"].notna().sum()} entries with values for place_2.')

There are 9192 entries with values for place_2.


<IPython.core.display.Javascript object>

#### Observations:
- We are finished searching `info_2` for place values and can move on to searching the other `info_` columns for them.
- First, we will make a working copy of `info_parenth` as its contents are no longer in the original `info` column and we need to preserve that information for reference.

#### Making Copy of `info_parenth`

In [40]:
# Creating info_parenth_2 column
df["info_parenth_copy"] = df["info_parenth"]

<IPython.core.display.Javascript object>

#### Checking Other `info_` Columns  for Remaining `place_1` Values

In [41]:
%%time

# Columns to search
cols_lst = [
    'info_1',
    'info_3',
    'info_4',
    'info_5',
    'info_6',
    'info_7',
    'info_8',
    'info_9',
    'info_10',
    'info_11',
    'info_parenth_copy'
]

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for column in cols_lst:
    for nationality, country in nation_map.items():
        dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item == nationality or item == country or item.startswith(nationality + ' ') or item.startswith(country + ' '):
                df.loc[index, extract_to] = country
                df.loc[index, column] = (
                    df.loc[index, column].replace(nationality, "").strip().replace(country, "").strip()
                )

CPU times: total: 34.7 s
Wall time: 34.7 s


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [42]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

There are 206 remaining missing values for place_1.



<IPython.core.display.Javascript object>

#### Observations:
- That iteration captured ~20 values.
- We will repeat it, but checking for values that are embedded.

#### Re-checking `info_3` through `info_11` and `info_parenth_copy` for `place_1` Embedded in Value

In [43]:
%%time

# Extract to column
extract_to = "place_1"

# For loop to extract nation data to place column
for column in cols_lst:
    for nationality, country in nation_map.items():
        dataframe = df[(df[extract_to].isna()) & (df[column].notna())]
        for index in dataframe.index:
            item = df.loc[index, column]
            if nationality + ' ' in item or country + ' ' in item or item.endswith(nationality) or item.endswith(country):
                df.loc[index, extract_to] = country
                df.loc[index, column] = (
                    df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
                )

CPU times: total: 26.8 s
Wall time: 26.8 s


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values for `place_1`

In [44]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')

# Checking a sample of rows with missing place_1
df[df["place_1"].isna()].sample(2)

There are 195 remaining missing values for place_1.



,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy
94330,21,Kurt Hansen,", 90, footballer[9]","https://en.wikipedia.org/wiki/Kurt_Hansen_(footballer,_born_1928)",2,2018,February,None,None,footballer[9],None,None,None,None,None,None,None,None,None,90.0,None,None,None,None
15189,21,Clifton Daniel,", 87, managing editor of , stroke.",https://en.wikipedia.org/wiki/Clifton_Daniel,6,2000,February,None,None,managing editor of,stroke,None,None,None,None,None,None,None,None,87.0,None,None,None,None


<IPython.core.display.Javascript object>

#### Obervations:
- We have completed the search for `place_1` and the remaining values do appear to be lacking that data.
- The next step is to check these columns for any `place_2` values.

#### Checking  for Remaining `place_2` Values in Other `info_` Columns

In [45]:
%%time

# Columns to search
cols_lst = [
    'info_1',
    'info_3',
    'info_4',
    'info_5',
    'info_6',
    'info_7',
    'info_8',
    'info_9',
    'info_10',
    'info_11',
    'info_parenth_copy'
]

# Extract to column
extract_to = "place_2"

# For loop to extract nation data to place column
for column in cols_lst:
    for nationality, country in nation_map.items():
        dataframe = df[(df['place_1'].notna()) & (df[extract_to].isna()) & (df[column].notna())]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item==nationality or item==country or item.startswith(nationality+ ' ') or item.startswith(country + ' '):
                df.loc[index, extract_to] = country
                df.loc[index, column] = (
                    df.loc[index, column].replace(nationality, "").strip().replace(country, '').strip()
                  )

CPU times: total: 7min 54s
Wall time: 7min 54s


<IPython.core.display.Javascript object>

#### Observations:
- We should stop our extraction of place values here, as searching for embedded `place_2` values is likely to generate erroneous values.
- For example, "English foremost expert on Poland" would result in Poland as a value for `place_2`.  We accepted this possibility for the relative few `place_1` values that were extracted from embedded nationality or country data, but we will not do that for `place_2`.
- Let us take a look at a sample of some entries with only `place_1` values, to confirm this decision.

#### Examining Sample of Rows with Only `place_1` Value

In [46]:
df[(df["place_1"].notna()) & df["place_2"].isna()].sample(20)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy
35863,8,Carol Barnes,", 63, British ITN news presenter, stroke.",https://en.wikipedia.org/wiki/Carol_Barnes,12,2008,March,None,None,ITN news presenter,stroke,None,None,None,None,None,None,None,None,63.0,None,United Kingdom of Great Britain and Northern Ireland,None,None
81988,8,Ken Gorgal,", 87, American football player .",https://en.wikipedia.org/wiki/Ken_Gorgal,22,2016,May,"Cleveland Browns, Chicago Bears",None,football player,None,None,None,None,None,None,None,None,None,87.0,None,United States of America,None,"Cleveland Browns, Chicago Bears"
94177,14,Ruud Lubbers,", 78, Dutch politician and diplomat, Prime Minister .",https://en.wikipedia.org/wiki/Ruud_Lubbers,45,2018,February,"1982 1994, United Nations High Commissioner for Refugees 2001 2005",None,politician and diplomat,Prime Minister,None,None,None,None,None,None,None,None,78.0,None,Netherlands,None,"1982 1994, United Nations High Commissioner for Refugees 2001 2005"
76308,2,Frank Callaway,", 69, Australian judge.",https://en.wikipedia.org/wiki/Frank_Callaway_(judge),4,2015,July,None,None,judge,None,None,None,None,None,None,None,None,None,69.0,None,Australia,None,None
120545,19,Leonard Sinclair Sparks,", 81, Canadian boxer.",https://en.wikipedia.org/wiki/Leonard_Sinclair_Sparks,24,2021,March,None,None,boxer,None,None,None,None,None,None,None,None,None,81.0,None,Canada,None,None
34417,25,Carole Hillard,", 71, American politician, Lieutenant Governor of South Dakota , pneumonia.",https://en.wikipedia.org/wiki/Carole_Hillard,1,2007,October,1995 2003,None,politician,Lieutenant Governor of South Dakota,pneumonia,None,None,None,None,None,None,None,71.0,None,United States of America,None,1995 2003
76379,6,Raymond Dean,", 68, American psychologist.",https://en.wikipedia.org/wiki/Raymond_Dean,11,2015,July,None,None,psychologist,None,None,None,None,None,None,None,None,None,68.0,None,United States of America,None,None
121699,24,John Coster-Mullen,", 74, American photographer and nuclear archaeologist, complications from amyotrophic lateral sclerosis.",https://en.wikipedia.org/wiki/John_Coster-Mullen,13,2021,April,None,None,photographer and nuclear archaeologist,complications from amyotrophic lateral sclerosis,None,None,None,None,None,None,None,None,74.0,None,United States of America,None,None
65708,2,Ian Mackley,", 71, British diplomat, High Commissioner to Ghana .",https://en.wikipedia.org/wiki/Ian_Mackley,2,2014,January,1996 2000,None,diplomat,High Commissioner to Ghana,None,None,None,None,None,None,None,None,71.0,None,United Kingdom of Great Britain and Northern Ireland,None,1996 2000
26442,17,Sir James Wilson,", 83, British army general.",https://en.wikipedia.org/wiki/James_Wilson_(British_Army_officer),2,2004,December,None,None,army general,None,None,None,None,None,None,None,None,None,83.0,None,United Kingdom of Great Britain and Northern Ireland,None,None


<IPython.core.display.Javascript object>

#### Observations:
- The sample, which can be run multiple times, validates the decision to not look for `place_2` values that aren't at the beginning of the column's string value.

#### Checking Counts of Missing Values for `place_1` and Number of Entries with Values for `place_2`

In [47]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')
print(f'There are {df["place_2"].notna().sum()} entries with values for place_2.')

There are 195 remaining missing values for place_1.

There are 11241 entries with values for place_2.


<IPython.core.display.Javascript object>

#### Observations:
- We are finished with extracting the place values, but we can search for "IRA" and "CIA" in `info` where `place_1` is missing and assign "Ireland" and "United States of America", respectively.  It is important not to remove these values from the original column as we might need them as "known_for" values.  Here, we are accepting the possibility that "CIA" could refer to an operative native to a location outside of the United States of America, but expect the possible impact to be trivial.
- It is time to save our dataframe and start a new notebook, before extracting "known for" and `cause_of_death` values.
- First, let us remove any remaining digits and nation or country values from the numbered `info_` and `info_parenth_copy` columns.
- We will repeat clearing out extra spaces afterward.

#### Assigning `place_1` for "CIA" and "IRA"

In [48]:
# Separate treatment of CIA and IRA for place_1
for index in df[df["place_1"].isna()].index:
    item = df.loc[index, "info"]
    if "CIA" in item:
        df.loc[index, "place_1"] = nation_map["US"]
    if "IRA" in item:
        df.loc[index, "place_1"] = nation_map["Irish"]

<IPython.core.display.Javascript object>

#### Final Counts of Missing Values for `place_1` and Number of Entries with Values for `place_2`

In [49]:
# Checking number of remaining missing values for place_1
print(f'There are {df["place_1"].isna().sum()} remaining missing values for place_1.\n')
print(f'There are {df["place_2"].notna().sum()} entries with values for place_2.')

There are 193 remaining missing values for place_1.

There are 11241 entries with values for place_2.


<IPython.core.display.Javascript object>

#### Observations:
- As expected, that iteration made a very small impact, capturing only 2 new `place_1` values.
- We have ~190 entries with missing `place_` from information.

### Removing Remaining Digits and Nationality/Country Values from `info_` Columns

#### List of Columns to Treat

In [50]:
# List of columns to treat
cols_lst = [
    "info_1",
    "info_2",
    "info_3",
    "info_4",
    "info_5",
    "info_6",
    "info_7",
    "info_8",
    "info_9",
    "info_10",
    "info_11",
    "info_parenth_copy",
]

<IPython.core.display.Javascript object>

#### Removing Digits

In [51]:
# Regular expression for parenthesis and its contents
pattern = r"\d"

# For loop to find indices of rows that have pattern
rows_combined = []
for column in cols_lst:
    dataframe = df[df[column].notna()]
    rows_to_check = rows_with_pattern(dataframe, column, pattern)
    rows_combined += rows_to_check

# Checking a sample of rows
df.loc[rows_combined, :].sample(2)

There are 0 rows with matching pattern in column 'info_1'.
There are 442 rows with matching pattern in column 'info_2'.
There are 2252 rows with matching pattern in column 'info_3'.
There are 1060 rows with matching pattern in column 'info_4'.
There are 69 rows with matching pattern in column 'info_5'.
There are 5 rows with matching pattern in column 'info_6'.
There are 0 rows with matching pattern in column 'info_7'.
There are 0 rows with matching pattern in column 'info_8'.
There are 0 rows with matching pattern in column 'info_9'.
There are 0 rows with matching pattern in column 'info_10'.
There are 0 rows with matching pattern in column 'info_11'.
There are 24403 rows with matching pattern in column 'info_parenth_copy'.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,info_parenth_copy
122268,11,Dan W. Brown,", 70, American politician, member of the Missouri House of Representatives .",https://en.wikipedia.org/wiki/Dan_W._Brown,6,2021,May,2008 2010 and Senate 2011 2019,None,politician,member of the Missouri House of Representatives,None,None,None,None,None,None,None,None,70.0,None,United States of America,None,2008 2010 and Senate 2011 2019
100890,26,Michel Legrand,", 86, French composer .",https://en.wikipedia.org/wiki/Michel_Legrand,40,2019,January,"Oscar, 1968, 1971, conductor and jazz pianist, 1983 winner , ,",None,composer,None,None,None,None,None,None,None,None,None,86.0,None,France,None,"Oscar, 1968, 1971, conductor and jazz pianist, 1983 winner , ,"


<IPython.core.display.Javascript object>

In [52]:
# For loop to extract digits
for column in cols_lst:
    for index in set(rows_combined):
        item = df.loc[index, column]
        if item:
            match = re.search(pattern, item)
            if match:
                df.loc[index, column] = re.sub(pattern, "", item)

# Rechecking number and example rows after treatment
# For loop to find indices of rows that have pattern
recheck_rows = []
for column in cols_lst:
    dataframe = df[df[column].notna()]
    rows_to_check = rows_with_pattern(dataframe, column, pattern)
    recheck_rows += rows_to_check

There are 0 rows with matching pattern in column 'info_1'.
There are 0 rows with matching pattern in column 'info_2'.
There are 0 rows with matching pattern in column 'info_3'.
There are 0 rows with matching pattern in column 'info_4'.
There are 0 rows with matching pattern in column 'info_5'.
There are 0 rows with matching pattern in column 'info_6'.
There are 0 rows with matching pattern in column 'info_7'.
There are 0 rows with matching pattern in column 'info_8'.
There are 0 rows with matching pattern in column 'info_9'.
There are 0 rows with matching pattern in column 'info_10'.
There are 0 rows with matching pattern in column 'info_11'.
There are 0 rows with matching pattern in column 'info_parenth_copy'.


<IPython.core.display.Javascript object>

#### Removing Any Remaining Matches with  `nation_map` Keys and Values

In [53]:
%%time

# For loop to extract remaining information matching items in nation_map
for column in cols_lst:
    dataframe = df[df[column].notna()]
    for nationality, country in nation_map.items():
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if nationality + ' ' in item or country + ' ' in item or item.endswith(nationality) or item.endswith(country):
                    df.loc[index, column] = item.replace(nationality, "").strip().replace(country,'').strip()

CPU times: total: 14min 24s
Wall time: 14min 25s


<IPython.core.display.Javascript object>

#### Removing Extra Spaces in `info_` Columns

In [54]:
%%time

# List of columns to treat
cols_lst = [
    "info_1",
    "info_2",
    "info_3",
    "info_4",
    "info_5",
    "info_6",
    "info_7",
    "info_8",
    "info_9",
    "info_10",
    "info_11",
    'info_parenth_copy'
]

# Dictionary of keys to find and values to replace keys
replace_dict = {"    ": ' ', "   ": ' ', '  ': ' '}

# For loop to find and replace characters in replace_dict in columns in cols_list
# and strip any leading or trailing periods or whitespace
for column in cols_lst:
    for key, value in replace_dict.items():
        for index in df[column].notna().index:
            item = df.loc[index, column]
            if item:
                df.loc[index, column] = item.replace(key, value).strip()

CPU times: total: 53.4 s
Wall time: 53.4 s


<IPython.core.display.Javascript object>

#### Observations:
- It's time to save our dataframe and start a new notebook before extracting "known for" and `cause_of_death` values.
- Exporting `nation_map` is also a good idea, at this time.

## Exporting Dataset to SQLite Database [wp_life_expect_clean3.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_clean3.db)

In [55]:
# Saving dataset in a SQLite database
conn = sql.connect("wp_life_expect_clean3.db")
df.to_sql("wp_life_expect_clean3", conn, index=False)

132652

<IPython.core.display.Javascript object>

## Saving nation_map to a Pickle File [nation_map.pkl](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/nation_map.pkl)

In [56]:
# Create a binary pickle file
f = open("nation_map.pkl", "wb")

# Write the dictionary to pickle file
pickle.dump(nation_map, f)

# close file
f.close()

# Chime notification when cell executes
chime.success()

<IPython.core.display.Javascript object>

# [Proceed to Data Cleaning Part 4](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean4_thanak_2022_06_23.ipynb)